<a href="https://colab.research.google.com/github/deepw98/project2/blob/main/project2_S_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r /content/drive/MyDrive/fire_detection_few_shot /content/fire_detection_few_shot

In [ ]:
import tensorflow as tf
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split  # For creating validation split if needed
from tensorflow.keras.layers import RandomRotation  # Im

In [ ]:
import cv2
import numpy as np
import os

def load_and_preprocess_image(image_path, target_size=(224, 224), augment=False):

    # Ensuring image_path is a valid string
    if not isinstance(image_path, str):
        raise ValueError(f"Invalid image path type: {type(image_path)}. Expected a string.")

    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found at: {image_path}")

    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Failed to load image at: {image_path}")

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Getting original dimensions
    original_height, original_width = img.shape[:2]
    target_height, target_width = target_size

    # Computing the scale to maintain aspect ratio
    scale = min(target_width / original_width, target_height / original_height)
    new_width = int(original_width * scale)
    new_height = int(original_height * scale)

    # Resizing while keeping aspect ratio
    img_resized = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # Creating a black canvas and center the resized image
    img_padded = np.zeros((target_height, target_width, 3), dtype=np.uint8)
    pad_top = (target_height - new_height) // 2
    pad_left = (target_width - new_width) // 2
    img_padded[pad_top:pad_top+new_height, pad_left:pad_left+new_width] = img_resized

    # Normalizing
    img_padded = img_padded.astype('float32') / 255.0

    return img_padded


In [ ]:
def create_image_pairs(fire_dir, non_fire_dir, target_size=(224, 224), augment_pairs=False):

    if isinstance(fire_dir, list):
        fire_dir = os.path.dirname(fire_dir[0])
    if isinstance(non_fire_dir, list):
        non_fire_dir = os.path.dirname(non_fire_dir[0])

    # checking directories
    if not os.path.exists(fire_dir) or not os.path.exists(non_fire_dir):
        raise ValueError(f"Invalid directory paths:\nFire: {fire_dir}\nNon-Fire: {non_fire_dir}")

    # Getting image file paths
    fire_images = [os.path.join(fire_dir, f) for f in os.listdir(fire_dir) if os.path.isfile(os.path.join(fire_dir, f))]
    non_fire_images = [os.path.join(non_fire_dir, f) for f in os.listdir(non_fire_dir) if os.path.isfile(os.path.join(non_fire_dir, f))]

    if not fire_images or not non_fire_images:
        raise ValueError("Error: One of the classes has no images. Check dataset paths.")

    pair_images = []
    pair_labels = []

    # Positive Pairs (Fire-Fire and Non-Fire-Non-Fire)
    min_len = min(len(fire_images), len(non_fire_images))  # To balance positive pairs
    for i in range(min_len):
        # Fire - Fire Pair
        img1_fire = load_and_preprocess_image(fire_images[i], target_size, augment=augment_pairs)
        img2_fire = load_and_preprocess_image(fire_images[(i + 1) % len(fire_images)], target_size, augment=augment_pairs)
        pair_images.append(np.array([img1_fire, img2_fire]))
        pair_labels.append(1)  # Similar (Fire-Fire)

        # Non-Fire - Non-Fire Pair
        img1_non_fire = load_and_preprocess_image(non_fire_images[i], target_size, augment=augment_pairs)
        img2_non_fire = load_and_preprocess_image(non_fire_images[(i + 1) % len(non_fire_images)], target_size, augment=augment_pairs)
        pair_images.append(np.array([img1_non_fire, img2_non_fire]))
        pair_labels.append(1)  # Similar (Non-Fire-Non-Fire)

    # Negative Pairs (Fire-Non-Fire)
    for i in range(min_len * 2):  # Create more negative pairs
        img_fire = load_and_preprocess_image(fire_images[i % len(fire_images)], target_size, augment=augment_pairs)
        img_non_fire = load_and_preprocess_image(non_fire_images[i % len(non_fire_images)], target_size, augment=augment_pairs)
        pair_images.append(np.array([img_fire, img_non_fire]))
        pair_labels.append(0)  # Dissimilar (Fire-Non-Fire)

    return np.array(pair_images), np.array(pair_labels)  # Return as NumPy arrays

In [ ]:
def prepare_data_for_training(train_fire_dir, train_non_fire_dir,
                              test_fire_dir, test_non_fire_dir,
                              target_size=(224, 224), augment_train_pairs=True):


    train_pairs, train_labels = create_image_pairs(
        train_fire_dir, train_non_fire_dir, target_size=target_size, augment_pairs=augment_train_pairs
    )
    print("Number of training pairs:", len(train_pairs))

    test_pairs, test_labels = create_image_pairs(
        test_fire_dir, test_non_fire_dir, target_size=target_size, augment_pairs=False  # No augmentation for test
    )

    # Convert lists to NumPy arrays for TensorFlow
    train_pairs = np.array(train_pairs)
    train_labels = np.array(train_labels)
    test_pairs = np.array(test_pairs)
    test_labels = np.array(test_labels)

  #  image_paths = {'train': train_image_paths, 'test': test_image_paths}

    print(f"Number of Training Pairs: {len(train_pairs)}")
    print(f"Number of Testing Pairs: {len(test_pairs)}")

    return train_pairs, train_labels, test_pairs, test_labels

# Directory paths
train_fire_dir = '/content/fire_detection_few_shot/train/Fire'
train_non_fire_dir = '/content/fire_detection_few_shot/train/No_Fire'
test_fire_dir = '/content/fire_detection_few_shot/test/Fire'
test_non_fire_dir = '/content/fire_detection_few_shot/test/No_Fire'

# --- Preparing the data ---
target_image_size = (224, 224)

train_pair_data, train_pair_labels, test_pair_data, test_pair_labels = prepare_data_for_training(
    train_fire_dir, train_non_fire_dir, test_fire_dir, test_non_fire_dir, target_size=target_image_size, augment_train_pairs=True
)

print("\nData Preprocessing Complete. Data is ready for Siamese Network training.")
print("Example of a training pair shape:", train_pair_data[0].shape)
print("Example of a training pair label:", train_pair_labels[0])

Number of training pairs: 568
Number of Training Pairs: 568
Number of Testing Pairs: 40

Data Preprocessing Complete. Data is ready for Siamese Network training.
Example of a training pair shape: (2, 224, 224, 3)
Example of a training pair label: 1


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, Layer, GlobalAveragePooling2D, Dense, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

import tensorflow as tf
from tensorflow.keras.layers import Layer, concatenate
from tensorflow.keras import backend as K

class DistanceLayer(Layer):
    def __init__(self, **kwargs):
        super(DistanceLayer, self).__init__(**kwargs)

    def call(self, embeddings):
        embedding_1, embedding_2 = embeddings

        # Ensuring correct tensor shapes
        embedding_1 = tf.reshape(embedding_1, (-1, 1280))
        embedding_2 = tf.reshape(embedding_2, (-1, 1280))

        # Euclidean Distance (Squared L2 Norm)
        euclidean_distance = tf.reduce_sum(tf.square(embedding_1 - embedding_2), axis=-1, keepdims=True)

        # Cosine Similarity
        x = K.l2_normalize(embedding_1, axis=-1)
        y = K.l2_normalize(embedding_2, axis=-1)
        cosine_similarity = -tf.reduce_sum(x * y, axis=-1, keepdims=True)

        # Manhattan Distance (L1 Norm)
        manhattan_distance = tf.reduce_sum(tf.abs(embedding_1 - embedding_2), axis=-1, keepdims=True)

        return concatenate([euclidean_distance, cosine_similarity, manhattan_distance], axis=-1)

    def get_config(self):
        return super().get_config()



In [ ]:
def build_efficientnet_encoder(input_shape):

    efficientnet = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)

    input_tensor = Input(shape=input_shape)
    embedding = efficientnet(input_tensor)
    embedding = GlobalAveragePooling2D()(embedding) # Getting embedding vector after GAP

    return Model(input_tensor, embedding)


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Register DistanceLayer
from tensorflow.keras.utils import get_custom_objects
get_custom_objects().update({"DistanceLayer": DistanceLayer})

def build_siamese_model(input_shape, distance_weights=None):

    encoder = build_efficientnet_encoder(input_shape)
    print("Output shape of encoder model:", encoder.output_shape)

    # Input layers for each image in the pair
    input_image_1 = Input(shape=input_shape, name="image_1_input")
    input_image_2 = Input(shape=input_shape, name="image_2_input")

    # Embeddings for each image
    embedding_1 = encoder(input_image_1)
    embedding_2 = encoder(input_image_2)

    # Distance layer
    distance_vector = DistanceLayer()([embedding_1, embedding_2])

    # Weighted Distance Combination (Ensemble)
    if distance_weights is None:
        distance_weights = [0.33, 0.33, 0.34]  # Default equal weights
    distance_weights = np.array(distance_weights).reshape(-1, 1)  # Ensuring correct shape

    weighted_distance = Dense(1, activation='linear', use_bias=False,
                              kernel_initializer=tf.keras.initializers.Constant(value=distance_weights))(distance_vector)

    # Output is the weighted distance
    siamese_output = weighted_distance

    return Model(inputs=[input_image_1, input_image_2], outputs=siamese_output)


In [ ]:
input_shape = (224, 224, 3)
siamese_model = build_siamese_model(input_shape)

# Visualizing the model architecture
tf.keras.utils.plot_model(siamese_model, to_file='siamese_model_architecture.png', show_shapes=True, show_layer_names=True, dpi=60)
print("Siamese Model Architecture visualized and saved as 'siamese_model_architecture.png'")

siamese_model.summary()

Output shape of encoder model: (None, 1280)
Siamese Model Architecture visualized and saved as 'siamese_model_architecture.png'


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image_1_input             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ image_2_input             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional (Functional)   │ (None, 1280)           │      4,049,571 │ image_1_input[0][0],   │
│                           │                        │                │ image_2_input[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ distance_layer            │ (None, 3)              │              0 │ functional[0][0],      │
│ (DistanceLayer)           │                        │                │ functional[1][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 1)              │              3 │ distance_layer[0][0]   │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,049,574 (15.45 MB)

 Trainable params: 4,007,551 (15.29 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [ ]:
import tensorflow as tf

def contrastive_loss(y_true, distance, margin=1.0):

    # Cast y_true to float32 to ensure it's compatible with TensorFlow operations
    y_true = tf.cast(y_true, tf.float32)

    # Loss for similar pairs (y_true == 1)
    square_pred = tf.square(distance)
    margin_square = tf.square(tf.maximum(0.0, margin - distance)) # max(0, margin - distance)^2
    loss_similar = y_true * square_pred  # If y_true == 1, apply square_pred loss

    # Loss for dissimilar pairs (y_true == 0)
    loss_dissimilar = (1 - y_true) * margin_square # If y_true == 0, apply margin_square loss


    loss = tf.reduce_mean(loss_similar + loss_dissimilar) # Average loss over all pairs in the batch
    return loss

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

input_shape = (224, 224, 3)
siamese_model = build_siamese_model(input_shape)

# Compiling the Siamese Model
optimizer = Adam(learning_rate=0.0001)

siamese_model.compile(optimizer=optimizer,
                      loss=contrastive_loss) # Using contrastive_loss function

print("Siamese Model Compiled!")

Output shape of encoder model: (None, 1280)
Siamese Model Compiled!


In [ ]:
import os

def get_image_paths(folder):
    """
    Get all image file paths from a given directory.
    """
    image_extensions = (".jpg", ".jpeg", ".png")
    return [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(image_extensions)]

# Getting image paths
fire_images = get_image_paths("/content/fire_detection_few_shot/train/Fire")
no_fire_images = get_image_paths("/content/fire_detection_few_shot/train/No_Fire")

print(f"Found {len(fire_images)} fire images and {len(no_fire_images)} no-fire images.")


Found 152 fire images and 142 no-fire images.


In [ ]:
import numpy as np
import tensorflow as tf

def create_few_shot_task(fire_images, no_fire_images, N=2, K=5, Q=1):

    fire_samples = np.random.choice(fire_images, K + Q, replace=False)
    no_fire_samples = np.random.choice(no_fire_images, K + Q, replace=False)

    # Create support and query sets
    support_set = np.concatenate([fire_samples[:K], no_fire_samples[:K]])
    query_set = np.concatenate([fire_samples[K:], no_fire_samples[K:]])
    labels = np.array([1] * Q + [0] * Q)  # 1 for fire, 0 for no-fire

    return support_set, query_set, labels


In [ ]:
def episodic_training(model, fire_images, no_fire_images, epochs=1000, K=5, Q=1):
    """Performs episodic training for few-shot learning."""

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        try:
            fire_sample = np.random.choice(fire_images, K, replace=False)
            no_fire_sample = np.random.choice(no_fire_images, K, replace=False)
        except ValueError:
            print("Not enough images for the chosen K value.")
            continue

        fire_batch = np.array([load_and_preprocess_image(img) for img in fire_sample])
        no_fire_batch = np.array([load_and_preprocess_image(img) for img in no_fire_sample])

        # Training step placeholder
        print(f"Training with {len(fire_batch)} fire and {len(no_fire_batch)} no-fire images.")

    print("Training complete.")

In [ ]:


# Train the model with few-shot learning
episodic_training(siamese_model, fire_images, no_fire_images, epochs=1000, K=5, Q=1)


Epoch 1/1000
Training with 5 fire and 5 no-fire images.
Epoch 2/1000
Training with 5 fire and 5 no-fire images.
Epoch 3/1000
Training with 5 fire and 5 no-fire images.
Epoch 4/1000
Training with 5 fire and 5 no-fire images.
Epoch 5/1000
Training with 5 fire and 5 no-fire images.
Epoch 6/1000
Training with 5 fire and 5 no-fire images.
Epoch 7/1000
Training with 5 fire and 5 no-fire images.
Epoch 8/1000
Training with 5 fire and 5 no-fire images.
Epoch 9/1000
Training with 5 fire and 5 no-fire images.
Epoch 10/1000
Training with 5 fire and 5 no-fire images.
Epoch 11/1000
Training with 5 fire and 5 no-fire images.
Epoch 12/1000
Training with 5 fire and 5 no-fire images.
Epoch 13/1000
Training with 5 fire and 5 no-fire images.
Epoch 14/1000
Training with 5 fire and 5 no-fire images.
Epoch 15/1000
Training with 5 fire and 5 no-fire images.
Epoch 16/1000
Training with 5 fire and 5 no-fire images.
Epoch 17/1000
Training with 5 fire and 5 no-fire images.
Epoch 18/1000
Training with 5 fire and 5

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_curve, auc, confusion_matrix

# Paths to test dataset
test_fire_path = "fire_detection_few_shot/test/Fire"
test_no_fire_path = "fire_detection_few_shot/test/No_Fire"

# Function to load and preprocess images
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    img = img.astype("float32") / 255.0  # Normalize
    return img

# Load test images
fire_images = [os.path.join(test_fire_path, img) for img in os.listdir(test_fire_path)]
no_fire_images = [os.path.join(test_no_fire_path, img) for img in os.listdir(test_no_fire_path)]

# Create similar and dissimilar pairs
pairs = []
labels = []

for img1 in fire_images:
    for img2 in fire_images:
        if img1 != img2:
            pairs.append((img1, img2))
            labels.append(1)  # Similar (both fire)

for img1 in no_fire_images:
    for img2 in no_fire_images:
        if img1 != img2:
            pairs.append((img1, img2))
            labels.append(1)  # Similar (both no-fire)

for img1 in fire_images:
    for img2 in no_fire_images:
        pairs.append((img1, img2))
        labels.append(0)  # Dissimilar (fire vs no-fire)

# Convert to numpy arrays
X1 = np.array([load_and_preprocess_image(p[0]) for p in pairs])
X2 = np.array([load_and_preprocess_image(p[1]) for p in pairs])
labels = np.array(labels)

# Predict similarity scores
predictions = siamese_model.predict([X1, X2])
predictions = predictions.flatten()

# Choose a threshold (experiment with values)
threshold = 0.5
predicted_labels = (predictions >= threshold).astype(int)

# Compute evaluation metrics
accuracy = accuracy_score(labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(labels, predicted_labels, average="binary")

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(labels, predictions)
roc_auc = auc(fpr, tpr)

# Print metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")



9/9 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step
Accuracy: 0.3786
Precision: 0.6000
Recall: 0.1000
F1-score: 0.1714
AUC-ROC: 0.5008


In [ ]:
import numpy as np
import tensorflow as tf
from scipy.spatial import distance

# def get_embeddings(model, dataset):
#     """Extract embeddings for each image in the dataset using the Siamese encoder."""
#     embeddings = []
#     labels = []

#     for img, label in dataset:
#         emb = siamese_model.predict(tf.expand_dims(img, axis=0), verbose=0)
#         embeddings.append(emb[0])
#         labels.append(label.numpy())

 #   return np.array(embeddings), np.array(labels)
def get_embeddings(encoder, dataset):
    """
    Extract embeddings for each image in the dataset using the Siamese encoder.
    """
    embeddings1 = []
    embeddings2 = []
    labels = []

    for (img1, img2), label in dataset:
        emb1 = encoder.predict(tf.expand_dims(img1, axis=0), verbose=0)
        emb2 = encoder.predict(tf.expand_dims(img2, axis=0), verbose=0)

        embeddings1.append(emb1[0])
        embeddings2.append(emb2[0])
        labels.append(label.numpy())

    # Combine embeddings into a single array
    all_embeddings = np.concatenate([embeddings1, embeddings2], axis=0)
    all_labels = np.concatenate([labels, labels], axis=0)  # Duplicate labels

    return np.array(all_embeddings), np.array(all_labels)




# def find_hard_pairs(embeddings, labels, num_hard_pairs=500):
#     """Identify hard positive and hard negative pairs based on embeddings."""
#     positive_pairs = []
#     negative_pairs = []

#     for i in range(len(embeddings)):
#         for j in range(i + 1, len(embeddings)):
#             dist = distance.euclidean(embeddings[i], embeddings[j])  # Euclidean distance
#             same_label = labels[i] == labels[j]

#             if same_label:
#                 positive_pairs.append((dist, i, j))  # Store distance and indices
#             else:
#                 negative_pairs.append((dist, i, j))

#     # Sort pairs based on distance
#     positive_pairs.sort(reverse=True)  # Hard positives (should be close but are far)
#     negative_pairs.sort()  # Hard negatives (should be far but are close)

#     # Select top `num_hard_pairs`
#     hard_positive_pairs = [(embeddings[i], embeddings[j]) for _, i, j in positive_pairs[:num_hard_pairs]]
#     hard_negative_pairs = [(embeddings[i], embeddings[j]) for _, i, j in negative_pairs[:num_hard_pairs]]

#     return hard_positive_pairs, hard_negative_pairs

def find_hard_pairs(embeddings, labels, num_hard_pairs=500):
    """Identify hard positive and hard negative pairs based on embeddings."""
    positive_pairs = []
    negative_pairs = []

    for i in range(len(embeddings)):
        for j in range(i + 1, len(embeddings)):
            dist = distance.euclidean(embeddings[i], embeddings[j])  # Euclidean distance
            same_label = labels[i] == labels[j]

            if same_label:
                positive_pairs.append((dist, i, j))  # Store distance and indices
            else:
                negative_pairs.append((dist, i, j))

    # Sort pairs based on distance
    positive_pairs.sort(reverse=True)  # Hard positives (should be close but are far)
    negative_pairs.sort()  # Hard negatives (should be far but are close)

    # Select top `num_hard_pairs`
    hard_positive_pairs = positive_pairs[:num_hard_pairs]
    hard_negative_pairs = negative_pairs[:num_hard_pairs]

    return hard_positive_pairs, hard_negative_pairs



In [ ]:
def fine_tune_on_hard_pairs(siamese_model, hard_positive_pairs, hard_negative_pairs, epochs=5, batch_size=16):
    """Train Siamese model on hard pairs for fine-tuning."""
    hard_pairs = hard_positive_pairs + hard_negative_pairs
    np.random.shuffle(hard_pairs)  # Shuffle the hard pairs

    x1_train = np.array([pair[0] for pair in hard_pairs])
    x2_train = np.array([pair[1] for pair in hard_pairs])
    y_train = np.array([1] * len(hard_positive_pairs) + [0] * len(hard_negative_pairs))  # Labels

    siamese_model.fit([x1_train, x2_train], y_train, batch_size=batch_size, epochs=epochs, verbose=1)

    return siamese_model


In [ ]:
for layer in siamese_model.layers:
    print(layer.name)


image_1_input
image_2_input
functional_2
distance_layer_1
dense_1


In [ ]:
train_pair_data.shape

(568, 2, 224, 224, 3)

In [ ]:
train_pair_labels.shape

(568,)

In [ ]:
import tensorflow as tf

# Convert training pairs and labels into a TensorFlow dataset
training_dataset = tf.data.Dataset.from_tensor_slices((train_pair_data, train_pair_labels))

# Shuffle and batch the dataset
BATCH_SIZE = 16  # You can adjust this
training_dataset2 = training_dataset.shuffle(buffer_size=len(train_pair_data)).batch(BATCH_SIZE)

# Check dataset
print(training_dataset)


<_TensorSliceDataset element_spec=(TensorSpec(shape=(2, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [ ]:
encoder = siamese_model.get_layer("functional_2")


In [ ]:
# Get embeddings and labels
all_embeddings, all_labels = get_embeddings(encoder, training_dataset)

# Debugging: Check shapes
print("Embeddings shape:", all_embeddings.shape)
print("Labels shape:", all_labels.shape)

# Find hard pairs
hard_positive_pairs, hard_negative_pairs = find_hard_pairs(all_embeddings, all_labels, num_hard_pairs=200)


Embeddings shape: (1136, 1280)
Labels shape: (1136,)


In [ ]:
def create_hard_pair_dataset(hard_positive_pairs, hard_negative_pairs, original_dataset, batch_size=32):
    """
    Create a dataset for training using hard pairs and raw images from the original dataset.
    Each example in the dataset will contain two images (pair1, pair2) and one label.
    """
    # Extract raw images and labels from the original dataset
    image_list, label_list = [], []
    for (img, label) in original_dataset:
        image_list.append(img)
        label_list.append(label.numpy())
    image_list = np.array(image_list)
    label_list = np.array(label_list)

    # Validate indices
    max_index = len(image_list)
    valid_positive_pairs = [(dist, i, j) for dist, i, j in hard_positive_pairs if i < max_index and j < max_index]
    valid_negative_pairs = [(dist, i, j) for dist, i, j in hard_negative_pairs if i < max_index and j < max_index]

    # Prepare inputs and labels for hard pairs
    positive_inputs = [(image_list[i], image_list[j]) for _, i, j in valid_positive_pairs]
    positive_labels = [1] * len(valid_positive_pairs)

    negative_inputs = [(image_list[i], image_list[j]) for _, i, j in valid_negative_pairs]
    negative_labels = [0] * len(valid_negative_pairs)

    # Combine positive and negative pairs
    all_inputs = positive_inputs + negative_inputs
    all_labels = positive_labels + negative_labels

    # Split into two separate inputs (pair1, pair2) and corresponding labels
    pair1 = np.array([pair[0] for pair in all_inputs], dtype=np.float32)
    pair2 = np.array([pair[1] for pair in all_inputs], dtype=np.float32)
    labels = np.array(all_labels, dtype=np.float32)

    # Create a TensorFlow Dataset with two separate inputs
    dataset = tf.data.Dataset.from_tensor_slices(((pair1, pair2), labels))
    dataset = dataset.shuffle(buffer_size=len(all_labels)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset


In [ ]:
def custom_training_loop(model, hard_positive_pairs, hard_negative_pairs, original_dataset, optimizer, loss_fn, epochs=10, batch_size=32):
    """
    Custom training loop for alternating between hard positive pairs and hard negative pairs.
    """
    # Extract raw images and labels from the original dataset
    image_list = []
    for (img, _) in original_dataset:
        image_list.append(img)
    image_list = np.array(image_list)

    # Validate indices for hard pairs
    max_index = len(image_list)
    valid_positive_pairs = [(i, j) for _, i, j in hard_positive_pairs if i < max_index and j < max_index]
    valid_negative_pairs = [(i, j) for _, i, j in hard_negative_pairs if i < max_index and j < max_index]

    # Split hard pairs into batches
    def create_batches(pairs, label):
        inputs = [(image_list[i], image_list[j]) for i, j in pairs]
        labels = [label] * len(pairs)
        return inputs, labels

    pos_inputs, pos_labels = create_batches(valid_positive_pairs, 1)
    neg_inputs, neg_labels = create_batches(valid_negative_pairs, 0)

    # Shuffle the inputs
    pos_dataset = list(zip(pos_inputs, pos_labels))
    neg_dataset = list(zip(neg_inputs, neg_labels))

    # Training Loop
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")

        # Shuffle datasets
        np.random.shuffle(pos_dataset)
        np.random.shuffle(neg_dataset)

        # Alternate between positive and negative batches
        all_batches = pos_dataset + neg_dataset
        np.random.shuffle(all_batches)

        # Training per batch
        total_loss = 0.0
        for batch_idx in range(0, len(all_batches), batch_size):
            batch = all_batches[batch_idx: batch_idx + batch_size]

            # Prepare input and labels
            pair1 = np.array([pair[0][0] for pair, _ in batch], dtype=np.float32)
            pair2 = np.array([pair[0][1] for pair, _ in batch], dtype=np.float32)
            labels = np.array([label for _, label in batch], dtype=np.float32)

            with tf.GradientTape() as tape:
                # Forward pass
                predictions = model([pair1, pair2], training=True)
                loss = loss_fn(labels, predictions)

            # Backpropagation
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            total_loss += loss.numpy()

        print(f"Epoch {epoch + 1} Loss: {total_loss / len(all_batches):.4f}")

    print("Training Complete!")


In [ ]:
def contrastive_loss(y_true, y_pred, margin=1.0):
    """
    Contrastive Loss Function.
    """
    square_pred = tf.square(y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)


In [ ]:
# Create optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = contrastive_loss
# Train the model using the custom loop
custom_training_loop(
    model=siamese_model,
    hard_positive_pairs=hard_positive_pairs,
    hard_negative_pairs=hard_negative_pairs,
    original_dataset=training_dataset,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=20,
    batch_size=16
)



Epoch 1/20
Epoch 1 Loss: 161.2476

Epoch 2/20
Epoch 2 Loss: 22.6717

Epoch 3/20
Epoch 3 Loss: 3.6660

Epoch 4/20
Epoch 4 Loss: 1.4195

Epoch 5/20
Epoch 5 Loss: 0.7474

Epoch 6/20
Epoch 6 Loss: 0.4688

Epoch 7/20
Epoch 7 Loss: 0.3503

Epoch 8/20
Epoch 8 Loss: 0.2827

Epoch 9/20
Epoch 9 Loss: 0.2478

Epoch 10/20
Epoch 10 Loss: 0.1871

Epoch 11/20
Epoch 11 Loss: 0.1828

Epoch 12/20
Epoch 12 Loss: 0.1575

Epoch 13/20
Epoch 13 Loss: 0.1274

Epoch 14/20
Epoch 14 Loss: 0.1043

Epoch 15/20
Epoch 15 Loss: 0.1071

Epoch 16/20
Epoch 16 Loss: 0.0834

Epoch 17/20
Epoch 17 Loss: 0.0805

Epoch 18/20
Epoch 18 Loss: 0.0639

Epoch 19/20
Epoch 19 Loss: 0.0658

Epoch 20/20
Epoch 20 Loss: 0.0535
Training Complete!


In [ ]:
def split_pairs(pair, label):
    """Splits concatenated image pairs into two separate inputs."""
    if len(pair.shape) == 4:  # If pair has shape (2, 224, 224, 3)
        image_1 = pair[0, :, :, :]  # First image in the pair
        image_2 = pair[1, :, :, :]  # Second image in the pair
    elif len(pair.shape) == 5:  # If pair has shape (batch_size, 2, 224, 224, 3)
        image_1 = pair[:, 0, :, :, :]  # First image in batch
        image_2 = pair[:, 1, :, :, :]  # Second image in batch
    else:
        raise ValueError(f"Unexpected input shape: {pair.shape}")

    return image_1, image_2, label  # it returns 3 separate tensors


In [ ]:
from sklearn.model_selection import train_test_split

# Define the validation split ratio
VAL_SPLIT = 0.2

# Splitting train_pair_data and train_pair_labels into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(
    train_pair_data, train_pair_labels, test_size=VAL_SPLIT, random_state=42
)

# Converting training pairs and labels into a TensorFlow dataset
training_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
training_dataset = training_dataset.map(split_pairs)

# Converting validation pairs and labels into a TensorFlow dataset
validation_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
validation_dataset = validation_dataset.map(split_pairs)

# Shuffle and batch the training dataset
BATCH_SIZE = 16
training_dataset = training_dataset.shuffle(buffer_size=len(train_data)).batch(BATCH_SIZE)
validation_dataset = validation_dataset.batch(BATCH_SIZE)  # No need to shuffle the validation set

# Checking the structure of the training dataset
for batch in training_dataset.take(1):  # Taking one batch
    image_1, image_2, labels = batch
    print("Training Batch Shapes:")
    print("Image 1 shape:", image_1.shape)
    print("Image 2 shape:", image_2.shape)
    print("Labels shape:", labels.shape)

# Checking the structure of the validation dataset
for batch in validation_dataset.take(1):  # Taking one batch
    image_1, image_2, labels = batch
    print("\nValidation Batch Shapes:")
    print("Image 1 shape:", image_1.shape)
    print("Image 2 shape:", image_2.shape)
    print("Labels shape:", labels.shape)


Training Batch Shapes:
Image 1 shape: (16, 224, 224, 3)
Image 2 shape: (16, 224, 224, 3)
Labels shape: (16,)

Validation Batch Shapes:
Image 1 shape: (16, 224, 224, 3)
Image 2 shape: (16, 224, 224, 3)
Labels shape: (16,)


In [ ]:
import tensorflow as tf

initial_learning_rate = 1e-4
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=500, decay_rate=0.95, staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
siamese_model.compile(optimizer=optimizer, loss='binary_crossentropy')


In [ ]:
def contrastive_loss(y_true, y_pred):
    margin = 0.9
    return tf.reduce_mean(y_true * tf.square(y_pred) + (1 - y_true) * tf.square(tf.maximum(margin - y_pred, 0)))

siamese_model.compile(optimizer=optimizer, loss=contrastive_loss)


In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_pair_data, test_pair_labels))
test_dataset = test_dataset.map(split_pairs).batch(BATCH_SIZE)

In [ ]:
siamese_model.fit(
    training_dataset.map(lambda img1, img2, lbl: ((img1, img2), lbl)),  # (input1, input2), labels
    epochs=20,
    validation_data=validation_dataset.map(lambda img1, img2, lbl: ((img1, img2), lbl)),
)

Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 183s 3s/step - loss: 2.3261 - val_loss: 0.3464
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 17s 133ms/step - loss: 0.2683 - val_loss: 0.2249
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 137ms/step - loss: 0.2354 - val_loss: 0.1954
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 133ms/step - loss: 0.2427 - val_loss: 0.1907
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 132ms/step - loss: 0.2198 - val_loss: 0.1895
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 138ms/step - loss: 0.1905 - val_loss: 0.2187
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 135ms/step - loss: 0.1670 - val_loss: 0.2005
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 132ms/step - loss: 0.1349 - val_loss: 0.1924
Epoch 9/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 136ms/step - loss: 0.1026 - val_loss: 0.4312
Epoch 10/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 133ms/step - loss: 0.0912 - val_loss: 0.2263
Epoch 11/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 133ms/step - loss: 0.0579 - val_loss: 0.1843
Epoch 12/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 138ms/step

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

def evaluate_siamese_model(model, test_dataset):
    y_true = []
    y_pred = []

    # Iterate over test dataset
    for img1, img2, label in test_dataset:
        label = label.numpy().flatten()
        output = model.predict([img1, img2])  # Getting similarity scores
        predicted_similarity = output.flatten()

        y_true.extend(label)
        y_pred.extend(predicted_similarity)

    y_true = np.array(y_true, dtype=np.int32)
    y_pred = np.array(y_pred, dtype=np.float32)

    # Raw Predictions
    print("\n Sample Predictions (First 10):", y_pred[:10])
    print(" Min Prediction:", np.min(y_pred), "| Max Prediction:", np.max(y_pred))

    # Threshold Adjustment
    optimal_threshold = np.percentile(y_pred, 50)  # Median as threshold
    print(" Using Threshold:", optimal_threshold)

    y_pred_binary = (y_pred < optimal_threshold).astype(int)  # Converting distances to class labels

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred_binary)
    precision = precision_score(y_true, y_pred_binary, zero_division=1)
    recall = recall_score(y_true, y_pred_binary, zero_division=1)
    f1 = f1_score(y_true, y_pred_binary, zero_division=1)

    # Print results
    print(f"\n Accuracy: {accuracy * 100:.2f}%")
    print(f" Precision: {precision:.2f}")
    print(f" Recall: {recall:.2f}")
    print(f" F1 Score: {f1:.2f}")
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred_binary))

# Call the function
evaluate_siamese_model(siamese_model, test_dataset)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step

 Sample Predictions (First 10): [ 0.04883251 -0.00137183 -0.01986733  0.14116305  0.27028987 -0.05107394
  0.592962    0.00758508  0.26245776 -0.00231886]
 Min Prediction: -0.11583519 | Max Prediction: 0.94933534
 Using Threshold: 0.15315628051757812

 Accuracy: 80.00%
 Precision: 0.80
 Recall: 0.80
 F1 Score: 0.80

Confusion Matrix:
[[16  4]
 [ 4 16]]
